<a href="https://colab.research.google.com/github/kocelifk/GNN_Adding_Extra_Layer/blob/main/gnn_adding_extra_layers_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install torch-geometric

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class BasicGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BasicGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Adding extra layers
class ExtendedGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super(ExtendedGNN, self).__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()

        # Input layer
        self.convs.append(GCNConv(input_dim, hidden_dim))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))

        # Output layer
        self.convs.append(GCNConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)
        return F.log_softmax(x, dim=1)

# Example usage
input_dim = 16  # Example input feature dimension
hidden_dim = 32  # Example hidden layer dimension
output_dim = 2  # Example output dimension (number of classes)

# Create an instance of the extended GNN model
model = ExtendedGNN(input_dim, hidden_dim, output_dim, num_layers=4)

# Example data
x = torch.randn((10, input_dim))  # 10 nodes with input_dim features
edge_index = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                           [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]], dtype=torch.long)

# Forward pass
out = model(x, edge_index)
print(out)


tensor([[-0.5621, -0.8440],
        [-0.6055, -0.7892],
        [-0.6589, -0.7286],
        [-0.6780, -0.7086],
        [-0.6605, -0.7269],
        [-0.6365, -0.7532],
        [-0.6070, -0.7874],
        [-0.5636, -0.8419],
        [-0.5233, -0.8978],
        [-0.5278, -0.8914]], grad_fn=<LogSoftmaxBackward0>)


In [ ]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4132 sha256=c2c3186631842f22c656a0e77a0450aa270ac16a0817959926d3cbc6c760c21b
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torchviz import make_dot

class ExtendedGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super(ExtendedGNN, self).__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()

        # Input layer
        self.convs.append(GCNConv(input_dim, hidden_dim))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))

        # Output layer
        self.convs.append(GCNConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)
        return F.log_softmax(x, dim=1)

# Example usage
input_dim = 16  # Example input feature dimension
hidden_dim = 32  # Example hidden layer dimension
output_dim = 2  # Example output dimension (number of classes)

# Create an instance of the extended GNN model
model = ExtendedGNN(input_dim, hidden_dim, output_dim, num_layers=4)

# Example data
x = torch.randn((10, input_dim))  # 10 nodes with input_dim features
edge_index = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                           [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]], dtype=torch.long)

# Forward pass
out = model(x, edge_index)
print(out)

# Visualize the model architecture
model_viz = make_dot(out, params=dict(model.named_parameters()))
model_viz.render("extended_gnn", format="png")


tensor([[-0.7780, -0.6150],
        [-0.7744, -0.6180],
        [-0.8026, -0.5945],
        [-0.8111, -0.5877],
        [-0.8164, -0.5834],
        [-0.8857, -0.5318],
        [-0.9167, -0.5106],
        [-0.8617, -0.5489],
        [-0.8217, -0.5792],
        [-0.8015, -0.5954]], grad_fn=<LogSoftmaxBackward0>)


'extended_gnn.png'